In [97]:
import numpy as np
from scipy import optimize as opt
from matplotlib import pyplot as plt

# Newton's

In [98]:
def newtons_method(Df,D2f,x0,tol=1e-5,maxiters=50):
    for k in range(maxiters): 
        b = D2f(x0)@x0-Df(x0)
        x1 = np.linalg.solve(D2f(x0),b)
        if np.linalg.norm(x1-x0)<tol:
            converge=True
            break
        converge=False
        x0=x1
    return x1,converge,(k+1)

In [99]:
Rosenbrock = lambda x: 100*((x[1]-x[0]**2)**2)+(1-x[0])**2
def rosen_jacob(x):
    jacob = np.zeros(2)
    jacob[0] = 400*(x[0]**3)+2*x[0]-2-400*x[0]*x[1]
    jacob[1] = -200*(x[0])**2+200*x[1]
    return jacob
def hessian_jacob(x):
    hess = np.zeros((2,2))
    hess[0,0] = 1200*(x[0]**2)+2-400*x[1]
    hess[0,1] = -400*x[0]
    hess[1,0] = -400*x[0] 
    hess[1,1] = 200
    return hess
init1 = np.array([-2,2])
init2 = np.array([10,-10])
print('Newton Rosenbrock initial 1:', newtons_method(rosen_jacob,hessian_jacob,init1))
print('Newton Rosenbrock initial 2:', newtons_method(rosen_jacob,hessian_jacob,init2))

Newton Rosenbrock initial 1: (array([ 1.,  1.]), True, 6)
Newton Rosenbrock initial 2: (array([ 1.,  1.]), True, 5)


# Broyden

In [100]:
def broyden(Df,A0,x0,tol=1e-5,maxiters=150):
    for k in range(maxiters): 
        b = A0@x0-Df(x0)
        x1 = np.linalg.solve(A0,b)
        y = Df(x1)-Df(x0)
        s = x1-x0
        A1 = A0+(y-A0@s)@(s/np.linalg.norm(s))
        if np.linalg.norm(s)<tol:
            converge=True
            break
        converge=False
        x0=x1
        A0=A1
    return x1,converge,(k+1)

In [101]:
f = lambda x: np.exp(x[0]-1)+np.exp(1-x[1])+(x[0]-x[1])**2
def Df(x):
    jacob = np.zeros(2)
    jacob[0] = np.exp(x[0]-1)+2*x[0]-2*x[1]
    jacob[1] = -np.exp(1-x[1])+2*x[1]-2*x[0]
    return jacob
J1=lambda x: np.array([2.*(x[0]-x[1])+np.exp(x[0]-1),-2.*(x[0]-x[1])-np.exp(1.-x[1])])
init1 = np.array([2,3])
init2 = np.array([3,2])
print('Broyden initial 1:', broyden(Df,H1(init1),init1))
print('Broyden initial 2:', broyden(Df,H1(init2),init2))

Broyden initial 1: (array([ 0.79611962,  1.20389324]), True, 18)
Broyden initial 2: (array([ 0.79614324,  1.20390998]), True, 112)


# BFGS

In [118]:
def BFGS(Df,A0,x0,tol=1e-5,maxiters=150):
    for k in range(maxiters): 
        b = A0@x0-Df(x0)
        x1 = np.linalg.solve(A0,b)
        y = np.array([Df(x1)-Df(x0)]).T
        s = np.array([x1-x0]).T
        A1 = A0+(y@y.T)/(y.T@s)-(A0@s@s.T@A0)/(s.T@A0@s)
        if np.linalg.norm(s)<tol:
            converge=True
            break
        converge=False
        x0=x1
        A0=A1
    return x1,converge,(k+1)      

In [119]:
f = lambda x: np.exp(x[0]-1)+np.exp(1-x[1])+(x[0]-x[1])**2
def Df(x):
    jacob = np.zeros(2)
    jacob[0] = np.exp(x[0]-1)+2*x[0]-2*x[1]
    jacob[1] = -np.exp(1-x[1])+2*x[1]-2*x[0]
    return jacob
J1=lambda x: np.array([2.*(x[0]-x[1])+np.exp(x[0]-1),-2.*(x[0]-x[1])-np.exp(1.-x[1])])
init1 = np.array([2,3])
init2 = np.array([3,2])
print('Broyden initial 1:', BFGS(Df,H1(init1),init1))
print('Broyden initial 2:', BFGS(Df,H1(init2),init2))

Broyden initial 1: (array([ 0.79611165,  1.20388835]), True, 7)
Broyden initial 2: (array([ 0.79611164,  1.20388835]), True, 8)


Note that Broyden's method does it in much fewer iterations.

# Gauss-Newton